In [ ]:
import torch
import torch.nn as nn
import numpy as np
from lib.model import RNN, DCN, Encoder, EncDec
import random
from lib.utils import Dataset,train_model, inference
from math import exp
from lib.generator import Shift, LorenzRandFGenerator
from torch.utils.tensorboard import SummaryWriter

SEED = 1234
DTYPE = torch.float64
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.backends.cudnn.benchmark=True
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [54]:
data_name = f'Shift'

train_size = 3000
test_size = 500

train_dataset = Dataset(*Shift({'path_len':8,'shift': 2}).generate(data_num=train_size), dtype=DTYPE, device=device)
test_dataset = Dataset(*Shift({'path_len':8,'shift': 2}).generate(data_num=test_size), dtype=DTYPE, device=device)

train_data = torch.utils.data.DataLoader(train_dataset, batch_size=128,drop_last=True)
test_data = torch.utils.data.DataLoader(test_dataset, batch_size=128,drop_last=True)

In [55]:
experiment_name = f'{data_name}_rnn'
rnn = RNN(input_dim=1, output_dim=1, hid_dim=32, activation='tanh').double().to(device)
rnn.load_state_dict(torch.load(f"saved_model/{experiment_name}/best_valid.pt"))
rnn.count_parameters()
train_model(name=experiment_name,model=rnn,train_data=train_data, test_data=test_data)
print(f'Best Valid Loss: {np.mean(inference(rnn, test_data)):.2e}')

Train Loss: 5.604e-02 	 Val. Loss: 1.145e-02 	 Best Loss: 5.604e-02 	 Current lr: 1.000e-03: | 0/1000 [00:00<?, ?it/s]

The model has 1,153 trainable parameters


Train Loss: 1.747e-06 	 Val. Loss: 1.792e-06 	 Best Loss: 1.747e-06 	 Current lr: 1.000e-05: | 1000/1000 [01:39<00:00, 10.09it/s]


Best Valid Loss: 1.79e-06


In [56]:
hess = torch.autograd.functional.hessian(lambda x: torch.sum(rnn(x)), train_dataset[0:1][0])
torch.norm(hess)

tensor(0.1852, device='cuda:0', dtype=torch.float64)

In [ ]:
experiment_name = f'{data_name}_cnn'
cnn = DCN(input_dim=1, output_dim=1, hid_dim=1, kernel_size=2, num_layers=6).double().to(device)
cnn.load_state_dict(torch.load(f"saved_model/{experiment_name}/best_valid.pt"))
cnn.count_parameters()
# train_model(name=experiment_name,model=cnn,train_data=train_data, test_data=test_data)
print(f'Best Valid Loss: {np.mean(inference(cnn, test_data)):.2e}')

In [ ]:
data_name = f'Shift'

train_size = 3000
test_size = 500

patch_size = 8

x, y = Shift({'path_len':128,'shift': 40}).generate(data_num=train_size)

x = x.reshape(train_size, -1,patch_size)
y = y.reshape(train_size, -1, patch_size)

train_dataset = Dataset(x,y, dtype=DTYPE, device=device)

xx, yy = Shift({'path_len':128,'shift': 40}).generate(data_num=test_size)

xx = xx.reshape(test_size, -1,patch_size)
yy = yy.reshape(test_size, -1, patch_size)

test_dataset = Dataset(xx,yy , dtype=DTYPE, device=device)

train_data = torch.utils.data.DataLoader(train_dataset, batch_size=128,drop_last=True)
test_data = torch.utils.data.DataLoader(test_dataset, batch_size=128,drop_last=True)

In [ ]:
experiment_name = f'{data_name}_transformer'
transformer = Encoder(8, 8,hid_dim=16,n_layers=2,n_heads=2, pf_dim=8, dropout=0.1,device=device,max_length = 16,pos_embed = True, param = 'full').double().to(device)
# transformer.load_state_dict(torch.load(f"saved_model/{experiment_name}/best_valid.pt"))
transformer.count_parameters()
train_model(name=experiment_name,model=transformer,train_data=train_data, test_data=test_data, epochs=10000)
print(f'Best Valid Loss: {np.mean(inference(transformer, test_data)):.2e}')

In [ ]:
experiment_name = f'{data_name}_encdec'
encdec = EncDec(1, 1, hid_dim=8, output_len=128).double().to(device)
encdec.load_state_dict(torch.load(f"saved_model/{experiment_name}/best_valid.pt"))
# train_model(name=experiment_name,model=encdec,train_data=train_data, test_data=test_data)
print(f'Best Valid Loss: {np.mean(inference(encdec, test_data)):.2e}')

In [ ]:
data_name = f'Lorentz'


train_size = 3000
test_size = 500

train_dataset = Dataset(*LorenzRandFGenerator({'path_len':128, 'n_init':train_size, 'K':1, 'J':6}).generate(data_num=train_size), dtype=DTYPE, device=device)
test_dataset = Dataset(*LorenzRandFGenerator({'path_len':128, 'n_init':test_size,'K':1, 'J':6}).generate(data_num=test_size), dtype=DTYPE, device=device)



train_data = torch.utils.data.DataLoader(train_dataset, batch_size=128,drop_last=True)
test_data = torch.utils.data.DataLoader(test_dataset, batch_size=128,drop_last=True)

In [ ]:
hess = torch.autograd.functional.hessian(lambda x: torch.sum(rnn(x)), train_dataset[0:1][0])
torch.norm(hess)

In [ ]:
experiment_name = f'{data_name}_rnn'
rnn = RNN(input_dim=1, output_dim=1, hid_dim=32, activation='tanh').double().to(device)
rnn.load_state_dict(torch.load(f"saved_model/{experiment_name}/best_valid.pt"))
rnn.count_parameters()
# train_model(name=experiment_name,model=rnn,train_data=train_data, test_data=test_data)
print(f'Best Valid Loss: {np.mean(inference(rnn, test_data)):.2e}')

In [ ]:
experiment_name = f'{data_name}_cnn'
cnn = DCN(input_dim=1, output_dim=1, hid_dim=16, kernel_size=2, num_layers=8, activation='tanh').double().to(device)
cnn.load_state_dict(torch.load(f"saved_model/{experiment_name}/best_valid.pt"))
cnn.count_parameters()
# train_model(name=experiment_name,model=cnn,train_data=train_data, test_data=test_data)
print(f'Best Valid Loss: {np.mean(inference(cnn, test_data)):.2e}')

In [ ]:
experiment_name = f'{data_name}_encdec'
encdec = EncDec(1, 1, hid_dim=64, output_len=128, activation='tanh').double().to(device)
# transformer.load_state_dict(torch.load(f"saved_model/{experiment_name}/best_valid.pt"))
train_model(name=experiment_name,model=encdec,train_data=train_data, test_data=test_data)
print(f'Best Valid Loss: {np.mean(inference(encdec, test_data)):.2e}')

In [47]:
class FindMaxDataset(torch.utils.data.Dataset):
    # Input is 1 2 8 9 0, outpu is 1 2 8 9 9  last position is the largetst value
    def __init__(self, num_of_data,seq_length, patch_size, dtype, device, minmax = 'max'):
        self.X, self.Y = self.generate_data(num_of_data,seq_length, patch_size, dtype, device, minmax)


    def __len__(self):
        return len(self.X)                   # return length

    def __getitem__(self, idx):
        return (self.X[idx], self.Y[idx])

    def generate_data(self,num_of_data,seq_length, patch_size, dtype, device, minmax):
        assert seq_length % patch_size == 0
        num_of_patches = seq_length // patch_size

        x = []
        y = []

        for i in range(num_of_data):
            seq = np.abs(np.random.randn(seq_length))
            # seq[-1] = 0
            x.append(torch.tensor(seq.reshape(-1,patch_size), dtype=dtype, device=device))
            if minmax == 'max':
                for i in range(1,seq_length):
                    seq[i] = np.max(seq[:i])
            elif minmax == 'min':
                seq[-1] = np.min(seq[:-1])
            else:
                raise Exception
            y.append(torch.tensor(seq.reshape(-1,patch_size), dtype=dtype, device=device))
            

        return x, y

In [58]:
train_dataset = FindMaxDataset(num_of_data=train_size,seq_length=8, 
                                patch_size=1, dtype=DTYPE, device=device, minmax='max')
test_dataset  = FindMaxDataset(num_of_data=test_size,seq_length=8, 
                                patch_size=1, dtype=DTYPE, device=device, minmax='max')

In [59]:
data_name = 'find_max'
experiment_name = f'{data_name}_rnn'
rnn = RNN(input_dim=1, output_dim=1, hid_dim=32, activation='tanh').double().to(device)
# rnn.load_state_dict(torch.load(f"saved_model/{experiment_name}/best_valid.pt"))
rnn.count_parameters()
train_model(name=experiment_name,model=rnn,train_data=train_data, test_data=test_data)
print(f'Best Valid Loss: {np.mean(inference(rnn, test_data)):.2e}')

Train Loss: 6.053e-02 	 Val. Loss: 4.394e-02 	 Best Loss: 6.053e-02 	 Current lr: 1.000e-03: | 2/1000 [00:00<01:34, 10.54it/s]

The model has 1,153 trainable parameters


Train Loss: 7.075e-07 	 Val. Loss: 6.483e-07 	 Best Loss: 7.075e-07 	 Current lr: 1.000e-04: | 1000/1000 [01:40<00:00,  9.94it/s]

Best Valid Loss: 6.48e-07


In [60]:
hess = torch.autograd.functional.hessian(lambda x: torch.sum(rnn(x)), train_dataset[0][0].unsqueeze(0))
torch.norm(hess)

tensor(1.3690, device='cuda:0', dtype=torch.float64)